## 250506 테스트
* text vectorDB 생성

In [1]:
import os
import re
import json
import jsonlines
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_milvus.vectorstores import Milvus
from uuid import uuid4
from langchain_community.chat_models import ChatClovaX
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import pandas as pd
import pytz

from datasets import Dataset
from datetime import timedelta
from operator import itemgetter
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import (
  AttributeInfo,
  StructuredQueryOutputParser,
  get_query_constructor_prompt
)
from langchain_teddynote.evaluator import GroundednessChecker
from langchain.retrievers.self_query.milvus import MilvusTranslator
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
import warnings
from langchain_core.runnables import chain

warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
embeddings = ClovaXEmbeddings(
    model='bge-m3'
)

In [12]:
URI = 'http://127.0.0.1:19530'

text_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='text_db_2'
)

In [13]:
docs = open_jsonl('./chunked_jsonl/text_semantic_per_80.jsonl')

In [7]:
docs

[Document(metadata={'type': 'text', 'page': 8, 'source': './raw_pdf_copy3/KISWeekly제1090호(20240628).pdf', 'summary': '- 국공채 시장은 약세로 시작, 외국인 매수세로 일부 회복.\n- 호주 5월 CPI가 전년 대비 4.0% 상승, 예상치 초과로 호주 국채 약세.\n- 국고채 20년 금리는 3.220%에 7천억 원 낙찰, 2조 1720억 원 응찰.\n- 28일 국고채 3년 최종호가는 3.182%, 10년물은 3.266%로 각각 3.8bp, 3.4bp 하락.\n- 국고 3년물은 지난주 대비 1.7bp 상승, 5년물은 0.5bp 상승, 10년물은 1.8bp 상승.\n- 국고 3/5년 스프레드는 1.8bp로 1.2bp 축소, 5/10년 스프레드는 6.5bp, 3/10년 스프레드는 8.3bp로 0.1bp 확대.', 'issue_date': '20240628'}, page_content='주 첫 영업일 국내 채권시장은 지난주 급격하게 하락한 금리를 되돌리며 약세로 출발했다. 장중 외 국인 매수세 지속으로 약세가 일부 되돌려졌으나, 전반적인 아시아 채권시장의 약세 분위기로 국내 채 권시장 또한 약세 우위를 보였다. 익영업일 미국 채권시장 강세의 영향으로 국고채 금리 또한 하락 출 발했다.'),
 Document(metadata={'type': 'text', 'page': 8, 'source': './raw_pdf_copy3/KISWeekly제1090호(20240628).pdf', 'summary': '- 국공채 시장은 약세로 시작, 외국인 매수세로 일부 회복.\n- 호주 5월 CPI가 전년 대비 4.0% 상승, 예상치 초과로 호주 국채 약세.\n- 국고채 20년 금리는 3.220%에 7천억 원 낙찰, 2조 1720억 원 응찰.\n- 28일 국고채 3년 최종호가는 3.182%, 10년물은 3.266%로 각각 3.8bp, 3.4bp 하락.\n- 국고 3년물은 지난주 대비 1.7bp 상

In [40]:
import time

def batch_add_documents(text_db, docs, batch_size=30, interval=65):
    total_docs = len(docs)
    for i in range(0, total_docs, batch_size):
        batch_docs = docs[i:i+batch_size]
        uuids = [str(uuid4()) for _ in range(len(batch_docs))]
        text_db.add_documents(documents=batch_docs, ids=uuids)
        if i + batch_size < total_docs:
            time.sleep(interval)

In [27]:
batch_add_documents(text_db, docs)

In [9]:
len(docs)

1675

In [28]:
URI = 'http://127.0.0.1:19530'

image_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='image_db'
)

In [29]:
docs = open_jsonl('./chunked_jsonl/image_semantic_per_80.jsonl')

In [33]:
batch_add_documents(image_db, docs)

In [39]:
table_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='table_db'
)

docs = open_jsonl('./chunked_jsonl/table_v7.jsonl')

batch_add_documents(table_db, docs)

In [35]:
len(docs)

2728

In [41]:
def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())

In [43]:
import os

def get_pdf_paths(directory):
    pdf_paths = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jsonl'):
                full_path = os.path.join(root, file)
                pdf_paths.append(full_path)

    return pdf_paths

# 사용 예시
# directory_path = "./parsed_pdf/result_v4"
directory_path = "./raptor"
file_paths = get_pdf_paths(directory_path)
file_paths

['./raptor\\KISWeekly제1090호(20240628)_raptor.jsonl',
 './raptor\\KISWeekly제1090호(20240629)_raptor.jsonl',
 './raptor\\KISWeekly제1091호(20240705)_raptor.jsonl',
 './raptor\\KISWeekly제1091호(20240706)_raptor.jsonl',
 './raptor\\KISWeekly제1092호(20240712)_raptor.jsonl',
 './raptor\\KISWeekly제1092호(20240713)_raptor.jsonl',
 './raptor\\KISWeekly제1093호(20240719)_raptor.jsonl',
 './raptor\\KISWeekly제1093호(20240720)_raptor.jsonl',
 './raptor\\KISWeekly제1094호(20240726)_raptor.jsonl',
 './raptor\\KISWeekly제1094호(20240727)_raptor.jsonl',
 './raptor\\KISWeekly제1095호(20240802)_raptor.jsonl',
 './raptor\\KISWeekly제1095호(20240803)_raptor.jsonl',
 './raptor\\KISWeekly제1096호(20240809)_raptor.jsonl',
 './raptor\\KISWeekly제1096호(20240810)_raptor.jsonl',
 './raptor\\KISWeekly제1097호(20240816)_raptor.jsonl',
 './raptor\\KISWeekly제1097호(20240817)_raptor.jsonl',
 './raptor\\KISWeekly제1098호(20240823)_raptor.jsonl',
 './raptor\\KISWeekly제1098호(20240824)_raptor.jsonl',
 './raptor\\KISWeekly제1099호(20240830)_raptor.j

In [44]:
docs = []
for file_path in file_paths:
    # JSONL 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 각 줄을 JSON으로 파싱
            data = json.loads(line)

            # Document 객체 생성 및 리스트에 추가
            doc = Document(
                page_content=data['page_content'],
                metadata=data['metadata']
            )
            docs.append(doc)

In [45]:
docs

[Document(metadata={'source': './raw_pdf_copy3/KISWeekly제1090호(20240628).pdf', 'issue_date': '20240628'}, page_content='주어진 문서는 최근 국내 및 아시아 채권시장, 주식시장, 그리고 외환시장의 동향을 요약하고 있습니다. \n\n1. **채권시장 동향**: \n   - 국내 채권시장은 주 초반에 금리 하락을 되돌리며 약세로 시작했습니다. \n   - 외국인 매수세로 인해 약세가 일부 완화되었으나, 아시아 채권시장의 전반적인 약세 분위기로 인해 국내 시장도 약세를 보였습니다.\n   - 미국 채권시장의 강세 영향으로 국고채 금리는 하락 출발했으나, 주 후반에는 다시 강세를 보이며 마감했습니다.\n\n2. **주식시장 동향**:\n   - KOSPI 지수는 전주 대비 13.56포인트 상승했습니다.\n   - 주요 지수 중 NIKKEI225는 2.56% 상승했으며, HSCEI는 1.68% 하락했습니다.\n\n3. **외환시장 동향**:\n   - 원/달러 환율은 지난주 대비 하락하여 원화 강세를 나타냈습니다. \n   - 유로, 엔화, 영국 파운드는 모두 미 달러 대비 약세를 보였습니다.\n   - 원/달러 스왑포인트는 1년 기준으로 -28.8원을 기록하며 환율 하락 전망을 시사했습니다.\n\n이 문서는 주식, 채권, 외환 시장의 주간 변동 사항을 종합적으로 다루고 있으며, 각 시장의 주요 지표와 변동 요인을 설명하고 있습니다.'),
 Document(metadata={'source': './raw_pdf_copy3/KISWeekly제1090호(20240628).pdf', 'issue_date': '20240628'}, page_content='주어진 문서는 최근 채권 시장의 동향과 경제 지표에 대한 내용을 다루고 있습니다. 주요 내용은 다음과 같습니다:\n\n1. **채권 시장 동향**: \n   - 국고채 20년물은 3.220%의 수익률로 7천억 원이 낙찰되었으며, 총

In [46]:
len(docs)

684

In [47]:
save_docs_to_jsonl(docs, 'chunked_jsonl/raptor.jsonl')

In [49]:
raptor_db = Milvus(
    embedding_function=embeddings,
    connection_args = {'uri': URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='raptor_db'
)

docs = open_jsonl('./chunked_jsonl/raptor.jsonl')

batch_add_documents(raptor_db, docs)